# Data analysis of Twitter API data

## Table of Contents
- [Introduction](#intro)
- [Part I - Gathering Data](#gathering)
- [Part II - Assessing Data](#assessing)
- [Part III - Cleaning Data](#cleaning)
- [Part IV - Data analysis](#analysis)

In [1]:
import os
import re
import pandas as pd
import numpy as np
import requests
import tweepy
import json
from timeit import default_timer as timer

%matplotlib inline


Bad key "text.kerning_factor" on line 4 in
c:\programdata\anaconda3\envs\udacity\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


<a id='intro'></a>
# Introduction
The dataset that you will be wrangling (and analyzing and visualizing) is the tweet archive of Twitter user *@dog_rates*, also known as *WeRateDogs*. *WeRateDogs* is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." *WeRateDogs* has over 4 million followers and has received international media coverage.

## Project Motivation
### Context
Goal: wrangle *WeRateDogs* Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.
### Data
**Enhanced Twitter Archive**

The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which I used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, I have filtered for tweets with ratings only (there are 2356).
I extracted this data programmatically, but I didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages (see below for more information on these) too. You'll need to assess and clean these columns if you want to use them for analysis and visualization.

## Key points
Key points to keep in mind when data wrangling for this project:

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* Cleaning includes merging individual pieces of data according to the rules of tidy data.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This unique rating system is a big part of the popularity of *WeRateDogs*.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.

<a id='gathering'></a>
# Part I - Gathering Data

The relevant data is retrieved by getting each of the three pieces of data as described below:

1. The *WeRateDogs* Twitter archive, download this file manually by clicking the following link: [twitter_archive_enhanced.csv](https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv)

2. The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file `image_predictions.tsv` is hosted on Udacity's servers.

3. Additionaly, each tweet's retweet count and favorite ("like") count at minimum is gathered. Using the tweet IDs in the *WeRateDogs* Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called `tweet_json.txt` file. Each tweet's JSON data is written to its own line. Then this .txt file is read line by line into a pandas DataFrame.

## 1. WeRateDogs Twitter archive
This table contains an information on the tweets from *WeRateDogs* Twitter archive up to August 1st, 2017.

In [2]:
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.head(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None


In [3]:
df_archive.shape

(2356, 17)

In [4]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

## 2. Image predictions of a dog breed in the tweet

This table contains prediction info, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet.

Created according to a neural network, can be downloaded from Udacity's servers.

In [3]:
URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
# Extract file name
file_name = URL.split('/')[-1]

In [ ]:
# Save the data
r = requests.get(URL)
if r.ok:    
    with open(file_name, mode='wb') as file:
        file.write(r.content)

In [4]:
# Read the tab separated file
df_predictions = pd.read_csv(file_name, sep='\t')
df_predictions.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [ ]:
df_predictions.info()

## 3. Connecting to Twitter API
At this step with help of [Tweepy](http://www.tweepy.org/query) Python library we wil query Twitter's API for additional data beyond the data already included in the WeRateDogs Twitter archive file. This additional data will include retweet count and favorite count.

In order to replicate this code cell, one needs to open a Twitter account.

[Tweepy API Documentation](http://docs.tweepy.org/en/v3.2.0/api.html#API)

In [ ]:
# Set up the connection to Twitter API (requires Twitter account)
consumer_key = os.getenv('TW_CONSUMER_KEY')
consumer_secret = os.getenv('TW_CONSUMER_SECRET')

access_token = os.getenv('TW_ACCESS_TOKEN')
access_secret = os.getenv('TW_ACCESS_SECRET')

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

# Note the handling of Twitter rate limit may extend the tweet query time
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

### Extract Tweet object data to a text file

In [ ]:
# Tweet IDs for which to gather additional data
tweet_ids = df_archive.tweet_id.values

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
failed_tweets_dict = {}
start = timer()

# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as file:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    # Rate limits are divided into 15 minute intervals
    for tweet_id in tweet_ids:
        count += 1
        # print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            # print("Success")
            json.dump(tweet._json, file)
            file.write('\n')
        except tweepy.TweepError as e:
            print(f'Tweet id - {tweet_id} - does not exist anymore.\n')
            failed_tweets_dict[tweet_id] = e
            pass
end = timer()
print(f'Finished. Total execution time: {round((end - start) / 60, 2)} minutes\n')
print(f'Number of missing tweets: {len(failed_tweets_dict.items())}\n')

### Investigate Tweet object data

The tweet object data is based on the tweet IDs in the `twitter-archive-enhanced.csv` and stored as a [Tweet JSON Object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/intro-to-tweet-json) in a text file.

In [ ]:
# Dictionary keys
tweet._json.keys()

In [ ]:
# Compare with the columns in the `twitter-archive-enhanced.csv`
df_archive.columns

In [ ]:
tweet._json

In [ ]:
tweet._json['full_text'][:139]

### Extract the number of Retweets and Favorite count
Code programmaticaly to count if a Tweet Object is a retweet and/or reply.

#### Load JSON Tweet objects created with Twitter API

In [5]:
# Create a list of tweet IDs dictionaries, retweets and replies
dict_list = []
retweet_list = [] 
reply_list = []

# Read .txt file as JSON file
with open('tweet_json.txt', 'r') as file:    
    for i, line in enumerate(file):       
        # Convert to Python dictionary
        data = json.loads(line)
        
        # Get tweet IDs if a tweet object is a retweet using regex
        if re.search(r'retweeted_status', line):
            retweet_list.append(data['id'])
        
        # Get reply tweet IDs
        if data['in_reply_to_status_id']:
            reply_list.append(data['id'])
            
        # Populate tweet dictionary
        dict_list.append({'tweet_id': data['id'],
                        'retweet_count': data['retweet_count'],
                        'favorite_count': data['favorite_count']                        
                        })
# Create a DataFrame with the the new parameters
df_stats = pd.DataFrame(dict_list, columns = ['tweet_id', 'retweet_count', 'favorite_count'])
df_stats.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,7730,36331
1,892177421306343426,5725,31313
2,891815181378084864,3787,23588
3,891689557279858688,7895,39641
4,891327558926688256,8515,37841


In [7]:
# Number or retweets, number of replies
len(retweet_list), len(reply_list)

(163, 77)

In [8]:
# Verify retweets in archive file
mask_retweets = df_archive.tweet_id.isin(retweet_list)
df_archive.loc[mask_retweets, ['tweet_id', 'text', 'retweeted_status_id']].sample(5)

,tweet_id,text,retweeted_status_id
250,845098359547420673,RT @dog_rates: This is Bungalo. She uses that ...,7.733088e+17
767,777641927919427584,RT @dog_rates: This is Arnie. He's a Nova Scot...,7.504293e+17
194,855245323840757760,RT @dog_rates: Meet George. He looks slightly ...,8.421635e+17
137,866094527597207552,RT @dog_rates: Here's a pupper before and afte...,8.378202e+17
589,799308762079035393,RT @dog_rates: I WAS SENT THE ACTUAL DOG IN TH...,7.743144e+17


In [9]:
# Verify replies in archive file
mask_retweets = df_archive.tweet_id.isin(reply_list)
df_archive.loc[mask_retweets, ['tweet_id', 'text', 'in_reply_to_user_id']].sample(5)

,tweet_id,text,in_reply_to_user_id
1905,674606911342424069,The 13/10 also takes into account this impecca...,4.196984e+09
346,831926988323639298,@UNC can confirm 12/10,2.068372e+07
1016,746906459439529985,"PUPDATE: can't see any. Even if I could, I cou...",4.196984e+09
274,840698636975636481,@0_kelvin_0 &gt;10/10 is reserved for puppos s...,8.405479e+17
1479,693582294167244802,Personally I'd give him an 11/10. Not sure why...,1.198989e+09


In [29]:
data

{'created_at': 'Sun Nov 15 22:32:08 +0000 2015',
 'id': 666020888022790149,
 'id_str': '666020888022790149',
 'full_text': 'Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj',
 'truncated': False,
 'display_text_range': [0, 131],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [],
  'urls': [],
  'media': [{'id': 666020881337073664,
    'id_str': '666020881337073664',
    'indices': [108, 131],
    'media_url': 'http://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg',
    'url': 'https://t.co/BLDqew2Ijj',
    'display_url': 'pic.twitter.com/BLDqew2Ijj',
    'expanded_url': 'https://twitter.com/dog_rates/status/666020888022790149/photo/1',
    'type': 'photo',
    'sizes': {'medium': {'w': 960, 'h': 720, 'resize': 'fit'},
     'thumb': {'w': 150, 'h': 150, 'resize': 'crop'},
     'large': {'w': 960, 'h': 720, 'resize': 'fit

<a id='assessing'></a>
# Part II - Assessing Data

After gathering each of the above pieces of data, they are assessed visually and programmatically for quality and tidiness issues. Detect and document at least:
    * (8) quality issues,
    * (2) tidiness issues.
    
To meet specifications, the issues that satisfy the Project Motivation (see the Key Points header in [Introduction](#intro)) must be assessed.

## Quality assessment summary
In `df_archive`:
> 1. Columns `'doggo', 'floofer', 'pupper', 'puppo'` show that they have no missing values, however it is clear that 'None' in these columns are actual missing values that are not read in correctly by software. 
    * Furthermore, there are also a lot of values that are not register for any the dog stage category (all with 'None'), this means that the dog stage is also not categorized or mentioned in the tweet.
    * And some observations have two different dog stages registered
    * Columns `'doggo', 'floofer', 'pupper', 'puppo'` are of *object* type, could be converted to *category* type. However due to that there two dog stages in one observations, it will be kept as a string.

> 2. Columns like `'tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id'`are of *float64* or *int* type, must be converted to str type to be as no mathematical operations are not expected to perform. The same must be done for `'tweet_id'` in other DataFrames.

> 3. Column `'timestamp'` is of *object* type, must be converted to *datetime* type. Furthermore, these dates are formatted differently than the `'created_at'` time data from Tweet Object.

> 4. Validity issues with numerator and denominator data - some values are not extracted properly according to the rating system. Also both must be coverted to *float* type

In `df_predictions` and `df_stats`:
> 6. Number of rows in predictions and archive data don't match:
    * There are some replies or retweets 
    * Also there are also some tweets without the dog rating
    * Some tweets are expired and must be removed
     
> 7. Drop redundant columns like:
    * `'source'` which just shows the utility used to post the Tweet. For example, Tweets from the Twitter website have a source value of web.
    * `'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp' ` after filtering out the retweet and reply tweets
    * 
    
> 8. Filter the `df_predictions` and add only correct and highest predictions to the master file
    * Correct prediction formatting

**Additional comments** 

> Consistency of column names with the [Tweet Object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object) attribute names in Tweet Data Dictionary. This may lead to confusion if someone else wants to extend the existing dataset. Rename the columns according to Tweet Object atributes:`'timestamp'` to `'created_at'` or with `'text'`  and `'full_text'`. However it is not implement in this exercise. 

## Tidiness assessment summary

1. In `df_archive` there are separate columns `'doggo', 'floofer', 'pupper', 'puppo'` that are one variable and must be melted together into a one column called 'dog_stage'

2. Merge into one dataset : Add the new retweet and favorite data from `df_stats` , valid predictions from `df_predictions` to twitter archive data in `df_archive` and create one file: `twitter-archive-master.csv`.

### Quality: Different number of observations

In [30]:
# Number of tweets dont match
df_archive.shape[0], df_predictions.shape[0], df_stats.shape[0]

(2356, 2075, 2331)

In [31]:
# Number of missing tweets
df_archive.shape[0] - df_stats.shape[0]

25

### Quality: Validity of the rating system data

In [ ]:
# Validity issues with numerator and denominator data
df_archive.rating_denominator.value_counts()

In [ ]:
df_archive.rating_numerator.value_counts()

### Quality: Missing values and appropriate data types

In [ ]:
df_archive.sample(3, random_state=42)

In [ ]:
df_archive.info()

### Quality: Datetime columns

In [33]:
# Date format from archive
df_archive.timestamp[0]

'2017-08-01 16:23:56 +0000'

In [32]:
# Date format from Tweet object is different (only relevant if we keep this info)
data['created_at']

'Sun Nov 15 22:32:08 +0000 2015'

### Tiddiness:  Dog stage columns

In [ ]:
# Check the unique values in dog stage columns
dog_stage_columns = ['doggo', 'floofer', 'pupper', 'puppo']

for col in dog_stage_columns:
    print(df_archive[col].unique())

In [ ]:
# Some of the dog stage columns have two values for the same observation
index_list = list()
for index, row in df_clean[dog_stage_columns].iterrows():
    if row.nunique() > 1:
        print(row.values)
        # Collect indices with multiple dog stages
        index_list.append(index)

In [ ]:
index_list

In [ ]:
pd.options.display.max_colwidth = 100
# Investigate tweet text extraction with Two stages 
df_clean.loc[index_list, dog_stage_columns + ['text']]

### Quality: Predictions table

In [10]:
df_predictions.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True


In [9]:
df_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [20]:
# Check for correct dog predictions
df_predictions.groupby('p1_dog').agg(['count', 'min', 'mean', 'max'])['p1_conf'].reset_index()

,p1_dog,count,min,mean,max
0,False,543,0.059033,0.540167,1.000000
1,True,1532,0.044333,0.613823,0.999956


In [21]:
df_predictions.groupby('p2_dog').agg(['count', 'min', 'mean', 'max'])['p2_conf'].reset_index()

,p2_dog,count,min,mean,max
0,False,522,1.011300e-08,0.11709,0.488014
1,True,1553,1.002880e-05,0.14047,0.467678


In [22]:
df_predictions.groupby('p3_dog').agg(['count', 'min', 'mean', 'max'])['p3_conf'].reset_index()

,p3_dog,count,min,mean,max
0,False,576,1.740170e-10,0.056893,0.255182
1,True,1499,1.434470e-06,0.061642,0.273419


In [14]:
# Print Top 10 predicted dogs labels
is_dog = df_predictions['p1_dog'] == True
df_predictions.loc[is_dog, 'p1'].value_counts().head(10)

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
chow                   44
Samoyed                43
toy_poodle             39
Pomeranian             38
malamute               30
Name: p1, dtype: int64

In [ ]:
# Check for duplicates tweets
df_predictions.tweet_id.duplicated().sum(), df_archive.tweet_id.duplicated().sum()

In [24]:
# Check if predictions contain information on the same tweets
df_predictions[['tweet_id']].append(df_archive[['tweet_id']]).tweet_id.unique().shape

(2356,)

In [27]:
df_archive.tweet_id.unique().shape[0], df_predictions.tweet_id.unique().shape[0]

(2356, 2075)

<a id='cleaning'></a>
# Part III - Cleaning Data

Clean each of the issues you documented while assessing.  The result should be a high quality and tidy master pandas DataFrame (or DataFrames, if appropriate). Again, the issues that satisfy the Project Motivation must be cleaned.

In [10]:
# Create a Copy of the original data before cleaning
df_clean = df_archive.copy()
df_stats_clean = df_stats.copy()
df_predictions_clean = df_predictions.copy()

### Quality Issues - Convert ID columns from integer to strings
**Define:** Convert IDs from integer to float data type df.astype('object') in `df_clean`, `df_stats_clean`, `df_predictions_clean`

**Code:**

In [8]:
# Convert IDs in all 3 data sources
id_columns = ['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id']

df_clean[id_columns] = df_clean[id_columns].astype('str')
df_stats_clean['tweet_id'] = df_stats_clean['tweet_id'].astype('str')
df_predictions_clean['tweet_id'] = df_predictions_clean['tweet_id'].astype('str')

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


**Test:**

In [9]:
assert df_clean[id_columns].dtypes.all() == 'object'
assert df_stats_clean['tweet_id'].dtypes == 'object'
assert df_predictions_clean['tweet_id'].dtypes == 'object'

In [10]:
# Verify tweet ID data types to be string (object in Pandas)
df_clean[id_columns].dtypes.all(), df_stats_clean['tweet_id'].dtypes, df_predictions_clean['tweet_id'].dtypes

(dtype('O'), dtype('O'), dtype('O'))

### Quality Issues - Filter Retweets and Replies
**Define:** Filter retweets and replies from `df_archive_clean` and `df_stats_clean`

**Code:**

In [11]:
# Number of rows before removing
df_clean.shape[0], df_stats_clean.shape[0]

(2356, 2331)

In [13]:
# Convert extracted IDs for retweets and replies from integer to str
# Retweet and reply tweet ids were axtracted from Twitter API data (for df_stats)
retweet_list = list(map(str, retweet_list))
reply_list = list(map(str, reply_list))
drop_list = list(set(retweet_list + reply_list))

In [14]:
# Number of rows to drop from df_stats_clean
len(drop_list)

240

In [15]:
# Create mask
mask_drop_tweets = df_stats_clean['tweet_id'].isin(drop_list)
# Index with a mask
df_stats_clean = df_stats_clean.loc[~mask_drop_tweets]

**Test:**

In [16]:
assert df_stats_clean['tweet_id'].isin(drop_list).any() == False

In [17]:
# Number of rows after
df_stats_clean.shape[0]

2091

**Code:**

As the `df_stats_clean` is now filtered for the retweets, replies and does not contain the tweets that have been removed in Twitter, we can filter the `df_clean` based on the `df_stats_clean` remaining observations. 

Another way to filter out the would be based on the `in_reply_to_status_id` and `retweeted_status_id` in the archive table but the would be redundant, as the current method deals with the problem already.

In [18]:
# Number of rows before removing
df_clean.shape[0], df_stats_clean.shape[0] 

(2356, 2091)

In [19]:
# Create mask and filter
mask_tweets = df_clean['tweet_id'].isin(df_stats_clean['tweet_id'])
df_clean = df_clean.loc[mask_tweets]

**Test:**

In [20]:
# Number of rows after
assert df_clean.shape[0] == df_stats_clean.shape[0]

In [22]:
# Number of rows after
df_clean.shape[0]

2091

### Quality Issues - Convert date strings to datetime
**Define:** Convert strings to datetime object with pd.to_datetime(). Note date format is different in `df_archive` and `df_stats`

**Code:**

In [114]:
# Convert str to datetime
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'], format='%Y-%m-%d %H:%M:%S +0000')

In [115]:
# df_stats_clean
#df_stats_clean['created_at'] = pd.to_datetime(df_stats['created_at'], format='%a %b %d %H:%M:%S +0000 %Y')
#assert df_stats_clean['created_at'].dtypes == np.dtype('datetime64[ns]')

**Test:**

In [116]:
assert df_clean['timestamp'].dtypes == np.dtype('datetime64[ns]')

In [117]:
df_clean[['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2091 entries, 0 to 2355
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  2091 non-null   datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 32.7 KB


### Tidiness Issues - Dog Stage Columns
**Define:** Combine dog stage columns into one column named `'dog_stage'` with str methods, replace 'None' with '', afterwards remove it with Series.str.strip()

**Code:**

In [119]:
dog_stage_columns = ['doggo', 'floofer', 'pupper', 'puppo']
df_clean[dog_stage_columns] = df_clean[dog_stage_columns].replace(to_replace='None', value='')

In [120]:
# Check if all the 'None' values are removed
for col in dog_stage_columns:
    assert df_clean[col].any() != 'None'

In [121]:
df_clean['dog_stage'] = df_clean['doggo'] + df_clean['floofer'] + df_clean['pupper'] + df_clean['puppo']

# Then format entries with multiple dog stages.
df_clean.loc[df_clean['dog_stage'] == 'doggopupper', 'dog_stage'] = 'doggo,pupper'
df_clean.loc[df_clean['dog_stage'] == 'doggopuppo', 'dog_stage'] = 'doggo,puppo'
df_clean.loc[df_clean['dog_stage'] == 'doggofloofer', 'dog_stage'] = 'doggo,floofer'

In [135]:
# Remove empty space character
df_clean['dog_stage'] = df_clean['dog_stage'].str.strip()

**Test:**

In [136]:
df_clean['dog_stage'].value_counts()

                 1756
pupper            221
doggo              71
puppo              23
floofer             9
doggo,pupper        9
doggo,puppo         1
doggo,floofer       1
Name: dog_stage, dtype: int64

**Define:** Convert str to category with df.astype('category')

**Code:**

In [126]:
df_clean['dog_stage'] = df_clean['dog_stage'].astype('category')

**Test:**

In [127]:
# Check if all the 'None' values are removed'
assert df_clean['dog_stage'].dtypes == 'category'

In [128]:
# Verify that the dog stage columns are converted properly
df_clean[dog_stage_columns].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2091 entries, 0 to 2355
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   doggo    2091 non-null   category
 1   floofer  2091 non-null   category
 2   pupper   2091 non-null   category
 3   puppo    2091 non-null   category
dtypes: category(4)
memory usage: 104.9 KB


In [138]:
df_clean.shape

(2091, 18)

### Quality Issues - Convert rating columns to float
**Define:** Convert numerator and denumenator from integer to float data type df.astype('float')

**Code:**

In [24]:
df_clean[['rating_numerator', 'rating_denominator']] = df_clean[['rating_numerator', 'rating_denominator']].astype('float')

**Test:**

In [26]:
assert df_clean[['rating_numerator', 'rating_denominator']].dtypes.all() == 'float'

In [27]:
df_clean[['rating_numerator', 'rating_denominator']].dtypes

rating_numerator      float64
rating_denominator    float64
dtype: object

### Quality Issues - Extract correct denominator values
**Define:** Read denominator values from tweet text column with Series.str.extract(regex)

**Code:**

In [31]:
ratings = df_clean.text.str.extract('(\d+)/(10)', expand=True)

In [38]:
ratings2 = df_clean.text.str.extract('((?:\d+\.)?\d+)\/(\d+)', expand=True)

In [48]:
ratings.iloc[:,0].value_counts()

12      484
10      436
11      414
13      284
9       153
8        98
7        51
14       40
5        34
6        32
3        19
4        15
2         9
1         4
0         1
27        1
420       1
75        1
1776      1
26        1
Name: 0, dtype: int64

In [47]:
df_clean.rating_numerator.value_counts()

12.0      484
10.0      435
11.0      413
13.0      283
9.0       153
8.0        98
7.0        52
14.0       39
5.0        34
6.0        32
3.0        19
4.0        16
2.0         9
1.0         5
24.0        1
27.0        1
84.0        1
0.0         1
420.0       1
75.0        1
80.0        1
60.0        1
44.0        1
26.0        1
144.0       1
88.0        1
121.0       1
99.0        1
204.0       1
1776.0      1
45.0        1
165.0       1
50.0        1
Name: rating_numerator, dtype: int64

### Quality Issues - Drop unnecessary columns
**Define:** Remove columns that are not need for the analysis with df.drop()

**Code:**

In [148]:
columns_to_drop = list(['source', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp', 'doggo', 'floofer', 'pupper', 'puppo'])

df_clean = df_clean.drop(columns_to_drop, axis='columns')

df_clean.shape

(2091, 8)

In [149]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2091 entries, 0 to 2355
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2091 non-null   object        
 1   timestamp           2091 non-null   datetime64[ns]
 2   text                2091 non-null   object        
 3   expanded_urls       2087 non-null   object        
 4   rating_numerator    2091 non-null   int64         
 5   rating_denominator  2091 non-null   int64         
 6   name                2091 non-null   object        
 7   dog_stage           2091 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 227.0+ KB


**Test:**

In [150]:
for col in columns_to_drop:
    assert col not in df_clean.columns

In [151]:
df_clean.columns

Index(['tweet_id', 'timestamp', 'text', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'dog_stage'],
      dtype='object')

### Quality Issues - Format Dog prediction labels
**Define:** use chained str atributes on string Series like: str.lower().str.replace().str.title()

**Code:**

In [36]:
# Format prediction columns
pred_id = ['p1', 'p2', 'p3']

df_predictions_clean[pred_id] = df_predictions_clean[pred_id].apply(lambda x: x.str.lower().str.replace('_', ' ').str.title())

**Test:**

In [54]:
# Verify that all words ar capitalized
assert df_predictions_clean[pred_id].applymap(lambda x: x[0].isupper() or x[1].isupper()).all().all()

# Verify that '_' is removed
assert df_predictions_clean[pred_id].apply(lambda x: x.str.contains('_')).all().all() == False

In [47]:
df_predictions_clean[pred_id].head(5)

,p1,p2,p3
0,Welsh Springer Spaniel,Collie,Shetland Sheepdog
1,Redbone,Miniature Pinscher,Rhodesian Ridgeback
2,German Shepherd,Malinois,Bloodhound
3,Rhodesian Ridgeback,Redbone,Miniature Pinscher
4,Miniature Pinscher,Rottweiler,Doberman


### Tidiness Issues - Combine data 
**Define:** Combine into one master Dataframe and store as 

**Code:**

In [52]:
df_predictions_clean[pred_id].apply(lambda x: x.str.contains('_')).all().all()

False

In [ ]:
for pred in pred_id:
    is_dog = df_predictions[f'{pred}_dog'] == True
    print(df_predictions.loc[is_dog, pred].value_counts())

In [61]:
df_predictions_clean.head(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel,0.465074,True,Collie,0.156665,True,Shetland Sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone,0.506826,True,Miniature Pinscher,0.074192,True,Rhodesian Ridgeback,0.072010,True


### Quality Issues - Select a Dog prediction 
**Define:** Choose a prediction of a dog label with df.apply(function) based on the following criteria that:
   * Firstly, a dog is actually predicted by neural network.
        * Secondly, probability of the prediction is higher than 20%.

**Code:**

In [93]:
# Function to select a prediction 


def select_prediction(df):    
    if df['p1_dog'] and df['p2_dog'] > 0.2:
        return df['p1']        
    elif df['p2_dog'] and df['p2_dog'] > 0.2:
        return df['p2']    
    elif df['p2_dog'] and df['p2_dog'] > 0.2:
        return df['p3']
    else:
        return 'Not a dog'
    

df_predictions_clean['Prediction'] = df_predictions_clean.apply(select_prediction, axis = 1)

**Test:**

In [94]:
df_predictions_clean['Prediction'].value_counts().head(10)

Not a dog             522
Golden Retriever      164
Labrador Retriever     99
Pembroke               88
Chihuahua              76
Pug                    51
Toy Poodle             42
Samoyed                37
Pomeranian             37
Chow                   35
Name: Prediction, dtype: int64

<a id='analysis'></a>
# Part IV - Analyzing Data
Storing, Analyzing, and Visualizing Data

Reporting of the project:
  1. Summary of data wrangling efforts are reported in `wrangle_report.html`.
  2. Summary of data analyses and visualizations are reported in `act_report.html`.

## Storing Data

Store the clean DataFrame(s) in a CSV file with the main one named `twitter_archive_master.csv`. If additional files exist because multiple tables are required for tidiness, name these files appropriately. Additionally, you may store the cleaned data in a SQLite database (which is to be submitted as well if you do).

## Visualisations